# List of Model Available 

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 5.1 MB/s 
     |████████████████████████████████| 182 kB 70.8 MB/s 


In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_rw_224',
 'coatnet_1_rw_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_rw_224',
 'coatnet_rmlp_1_rw_224',
 'coatnet_rmlp_2_rw_224',
 'coatnet_rmlp_nano_rw_224',
 'coatnext_nano_rw_224',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_atto',
 'convnext_atto_

In [ ]:
# Helper functions for evaluation
# Reference: Natural Adversarial Examples 
# https://github.com/hendrycks/natural-adv-examples

import numpy as np
import sklearn.metrics as sk
import sklearn.neighbors
import sklearn.ensemble
import time

recall_level_default = 0.95



def calib_err(confidence, correct, p='2', beta=100):
    # beta is target bin size
    idxs = np.argsort(confidence)
    confidence = confidence[idxs]
    correct = correct[idxs]
    bins = [[i * beta, (i + 1) * beta] for i in range(len(confidence) // beta)]
    bins[-1] = [bins[-1][0], len(confidence)]

    cerr = 0
    total_examples = len(confidence)
    for i in range(len(bins) - 1):
        bin_confidence = confidence[bins[i][0]:bins[i][1]]
        bin_correct = correct[bins[i][0]:bins[i][1]]
        num_examples_in_bin = len(bin_confidence)

        if num_examples_in_bin > 0:
            difference = np.abs(np.nanmean(bin_confidence) - np.nanmean(bin_correct))

            if p == '2':
                cerr += num_examples_in_bin / total_examples * np.square(difference)
            elif p == '1':
                cerr += num_examples_in_bin / total_examples * difference
            elif p == 'infty' or p == 'infinity' or p == 'max':
                cerr = np.maximum(cerr, difference)
            else:
                assert False, "p must be '1', '2', or 'infty'"

    if p == '2':
        cerr = np.sqrt(cerr)

    return cerr


def aurra(confidence, correct):
    conf_ranks = np.argsort(confidence)[::-1]  # indices from greatest to least confidence
    rra_curve = np.cumsum(np.asarray(correct)[conf_ranks])
    rra_curve = rra_curve / np.arange(1, len(rra_curve) + 1)  # accuracy at each response rate
    return np.mean(rra_curve)


def soft_f1(confidence, correct):
    wrong = 1 - correct

    # # the incorrectly classified samples are our interest
    # # so they make the positive class
    # tp_soft = np.sum((1 - confidence) * wrong)
    # fp_soft = np.sum((1 - confidence) * correct)
    # fn_soft = np.sum(confidence * wrong)

    # return 2 * tp_soft / (2 * tp_soft + fn_soft + fp_soft)
    return 2 * ((1 - confidence) * wrong).sum()/(1 - confidence + wrong).sum()


def tune_temp(logits, labels, binary_search=True, lower=0.2, upper=5.0, eps=0.0001):
    logits = np.array(logits)

    if binary_search:
        import torch
        import torch.nn.functional as F

        logits = torch.FloatTensor(logits)
        labels = torch.LongTensor(labels)
        t_guess = torch.FloatTensor([0.5*(lower + upper)]).requires_grad_()

        while upper - lower > eps:
            if torch.autograd.grad(F.cross_entropy(logits / t_guess, labels), t_guess)[0] > 0:
                upper = 0.5 * (lower + upper)
            else:
                lower = 0.5 * (lower + upper)
            t_guess = t_guess * 0 + 0.5 * (lower + upper)

        t = min([lower, 0.5 * (lower + upper), upper], key=lambda x: float(F.cross_entropy(logits / x, labels)))
    else:
        import cvxpy as cx

        set_size = np.array(logits).shape[0]

        t = cx.Variable()

        expr = sum((cx.Minimize(cx.log_sum_exp(logits[i, :] * t) - logits[i, labels[i]] * t)
                    for i in range(set_size)))
        p = cx.Problem(expr, [lower <= t, t <= upper])

        p.solve()   # p.solve(solver=cx.SCS)
        t = 1 / t.value

    return t


def get_measures(confidence, correct):
    rms = calib_err(confidence, correct, p='2')
    aurra_metric = aurra(confidence, correct)
    mad = calib_err(confidence, correct, p='1')  # secondary metric
    sf1 = soft_f1(confidence, correct)    # secondary metric

    return rms, aurra_metric, mad, sf1


def print_measures(rms, aurra_metric, mad, sf1, method_name='Baseline'):
    print('\t\t\t\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(100 * rms))
    print('AURRA (%): \t\t\t{:.2f}'.format(100 * aurra))
    # print('MAD Calib Error (%): \t\t{:.2f}'.format(100 * mad))
    # print('Soft F1 Score (%):   \t\t{:.2f}'.format(100 * sf1))


def show_calibration_results(confidence, correct):

    print('RMS Calib Error (%): \t{:.2f}'.format(
        100 * calib_err(confidence, correct, p='2')))

    print('AURRA (%): \t\t{:.2f}'.format(
        100 * aurra(confidence, correct)))

    # print('MAD Calib Error (%): \t\t{:.2f}'.format(
    #     100 * calib_err(confidence, correct, p='1')))

    # print('Soft F1-Score (%): \t\t{:.2f}'.format(
    #     100 * soft_f1(confidence, correct))



def stable_cumsum(arr, rtol=1e-05, atol=1e-08):
    """Use high precision for cumsum and check that final value matches sum
    Parameters
    ----------
    arr : array-like
        To be cumulatively summed as flat
    rtol : float
        Relative tolerance, see ``np.allclose``
    atol : float
        Absolute tolerance, see ``np.allclose``
    """
    out = np.cumsum(arr, dtype=np.float64)
    expected = np.sum(arr, dtype=np.float64)
    if not np.allclose(out[-1], expected, rtol=rtol, atol=atol):
        raise RuntimeError('cumsum was found to be unstable: '
                           'its last element does not correspond to sum')
    return out

def fpr_and_fdr_at_recall(y_true, y_score, recall_level=recall_level_default, pos_label=None):
    classes = np.unique(y_true)
    if (pos_label is None and
            not (np.array_equal(classes, [0, 1]) or
                     np.array_equal(classes, [-1, 1]) or
                     np.array_equal(classes, [0]) or
                     np.array_equal(classes, [-1]) or
                     np.array_equal(classes, [1]))):
        raise ValueError("Data is not binary and pos_label is not specified")
    elif pos_label is None:
        pos_label = 1.

    # make y_true a boolean vector
    y_true = (y_true == pos_label)

    # sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]

    # y_score typically has many tied values. Here we extract
    # the indices associated with the distinct values. We also
    # concatenate a value for the end of the curve.
    distinct_value_indices = np.where(np.diff(y_score))[0]
    threshold_idxs = np.r_[distinct_value_indices, y_true.size - 1]

    # accumulate the true positives with decreasing threshold
    tps = stable_cumsum(y_true)[threshold_idxs]
    fps = 1 + threshold_idxs - tps      # add one because of zero-based indexing

    thresholds = y_score[threshold_idxs]

    recall = tps / tps[-1]

    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)      # [last_ind::-1]
    recall, fps, tps, thresholds = np.r_[recall[sl], 1], np.r_[fps[sl], 0], np.r_[tps[sl], 0], thresholds[sl]

    cutoff = np.argmin(np.abs(recall - recall_level))

    return fps[cutoff] / (np.sum(np.logical_not(y_true)))   # , fps[cutoff]/(fps[cutoff] + tps[cutoff])

def get_measures(_pos, _neg, recall_level=recall_level_default):
    pos = np.array(_pos[:]).reshape((-1, 1))
    neg = np.array(_neg[:]).reshape((-1, 1))
    examples = np.squeeze(np.vstack((pos, neg)))
    labels = np.zeros(len(examples), dtype=np.int32)
    labels[:len(pos)] += 1

    auroc = sk.roc_auc_score(labels, examples)
    aupr = sk.average_precision_score(labels, examples)
    fpr = fpr_and_fdr_at_recall(labels, examples, recall_level)

    return auroc, aupr, fpr


def print_measures_old(auroc, aupr, fpr, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}'.format(int(100 * recall_level), 100 * fpr))
    print('AUROC: \t{:.2f}'.format(100 * auroc))
    print('AUPR:  \t{:.2f}'.format(100 * aupr))


def print_measures_with_std(aurocs, auprs, fprs, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}\t+/- {:.2f}'.format(int(100 * recall_level), 100 * np.mean(fprs), 100 * np.std(fprs)))
    print('AUROC: \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(aurocs), 100 * np.std(aurocs)))
    print('AUPR:  \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(auprs), 100 * np.std(auprs)))


def get_and_print_results(out_score, in_score, num_to_avg=1):

    aurocs, auprs, fprs = [], [], []
    #for _ in range(num_to_avg):
    #    out_score = get_ood_scores(ood_loader)
    measures = get_measures(out_score, in_score)
    aurocs.append(measures[0]); auprs.append(measures[1]); fprs.append(measures[2])

    auroc = np.mean(aurocs); aupr = np.mean(auprs); fpr = np.mean(fprs)
    #auroc_list.append(auroc); aupr_list.append(aupr); fpr_list.append(fpr)

    #if num_to_avg >= 5:
    #    print_measures_with_std(aurocs, auprs, fprs, method_name='Ours')
    #else:
    #    print_measures(auroc, aupr, fpr, method_name='Ours')
    return auroc, aupr, fpr

# Download Datasets

In [ ]:
# List of all robustness datasets: ImageNet-A, O, R, Validation, Sketch, Style

# Imagenet -a
!wget https://people.eecs.berkeley.edu/~hendrycks/imagenet-a.tar
!tar -xvf  './imagenet-a.tar'

# Imagenet -o
!wget https://people.eecs.berkeley.edu/~hendrycks/imagenet-o.tar
!tar -xvf  './imagenet-o.tar'

# Imagenet -r
!wget https://people.eecs.berkeley.edu/~hendrycks/imagenet-r.tar
!tar -xvf  './imagenet-r.tar'

# Imagenet validation
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
!mkdir imagenet_val
!tar -xvf './ILSVRC2012_img_val.tar' -C './imagenet_val'

# Imagenet sketch
!gdown https://drive.google.com/open?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA
!tar -xvf  './imagenet-sketch.zip'

# Imagenet style
!wget http://alisec-competition.oss-cn-shanghai.aliyuncs.com/xiaofeng/easy_robust/benchmark_datasets/imagenet-style.tar.gz
!tar -zxvf imagenet-style.tar.gz

/usr/local/lib/python3.8/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA
  warnings.warn(
Downloading...
From: https://drive.google.com/open?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA
To: /content/open?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA
71.0kB [00:00, 9.71MB/s]
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


# Imagenet : Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as trn
import torchvision.transforms.functional as trnF
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import numpy as np


PATH_TO_IMAGENET_A = "./imagenet-a"
PATH_TO_IMAGENET_O = "./imagenet-o"
PATH_TO_IMAGENET_VAL = "./imagenet_val"
PATH_TO_IMAGENET_R = "./imagenet-r"
PATH_TO_IMAGENET_SKETCH = "./imagenet-sketch"
PATH_TO_IMAGENET_STYLE = "./imagenet-style"

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

test_transform = trn.Compose(
    [trn.Resize(256), trn.CenterCrop(224), trn.ToTensor(), trn.Normalize(mean, std)])

all_wnids = ['n01440764', 'n01443537', 'n01484850', 'n01491361', 'n01494475', 'n01496331', 'n01498041', 'n01514668', 'n01514859', 'n01518878', 'n01530575', 'n01531178', 'n01532829', 'n01534433', 'n01537544', 'n01558993', 'n01560419', 'n01580077', 'n01582220', 'n01592084', 'n01601694', 'n01608432', 'n01614925', 'n01616318', 'n01622779', 'n01629819', 'n01630670', 'n01631663', 'n01632458', 'n01632777', 'n01641577', 'n01644373', 'n01644900', 'n01664065', 'n01665541', 'n01667114', 'n01667778', 'n01669191', 'n01675722', 'n01677366', 'n01682714', 'n01685808', 'n01687978', 'n01688243', 'n01689811', 'n01692333', 'n01693334', 'n01694178', 'n01695060', 'n01697457', 'n01698640', 'n01704323', 'n01728572', 'n01728920', 'n01729322', 'n01729977', 'n01734418', 'n01735189', 'n01737021', 'n01739381', 'n01740131', 'n01742172', 'n01744401', 'n01748264', 'n01749939', 'n01751748', 'n01753488', 'n01755581', 'n01756291', 'n01768244', 'n01770081', 'n01770393', 'n01773157', 'n01773549', 'n01773797', 'n01774384', 'n01774750', 'n01775062', 'n01776313', 'n01784675', 'n01795545', 'n01796340', 'n01797886', 'n01798484', 'n01806143', 'n01806567', 'n01807496', 'n01817953', 'n01818515', 'n01819313', 'n01820546', 'n01824575', 'n01828970', 'n01829413', 'n01833805', 'n01843065', 'n01843383', 'n01847000', 'n01855032', 'n01855672', 'n01860187', 'n01871265', 'n01872401', 'n01873310', 'n01877812', 'n01882714', 'n01883070', 'n01910747', 'n01914609', 'n01917289', 'n01924916', 'n01930112', 'n01943899', 'n01944390', 'n01945685', 'n01950731', 'n01955084', 'n01968897', 'n01978287', 'n01978455', 'n01980166', 'n01981276', 'n01983481', 'n01984695', 'n01985128', 'n01986214', 'n01990800', 'n02002556', 'n02002724', 'n02006656', 'n02007558', 'n02009229', 'n02009912', 'n02011460', 'n02012849', 'n02013706', 'n02017213', 'n02018207', 'n02018795', 'n02025239', 'n02027492', 'n02028035', 'n02033041', 'n02037110', 'n02051845', 'n02056570', 'n02058221', 'n02066245', 'n02071294', 'n02074367', 'n02077923', 'n02085620', 'n02085782', 'n02085936', 'n02086079', 'n02086240', 'n02086646', 'n02086910', 'n02087046', 'n02087394', 'n02088094', 'n02088238', 'n02088364', 'n02088466', 'n02088632', 'n02089078', 'n02089867', 'n02089973', 'n02090379', 'n02090622', 'n02090721', 'n02091032', 'n02091134', 'n02091244', 'n02091467', 'n02091635', 'n02091831', 'n02092002', 'n02092339', 'n02093256', 'n02093428', 'n02093647', 'n02093754', 'n02093859', 'n02093991', 'n02094114', 'n02094258', 'n02094433', 'n02095314', 'n02095570', 'n02095889', 'n02096051', 'n02096177', 'n02096294', 'n02096437', 'n02096585', 'n02097047', 'n02097130', 'n02097209', 'n02097298', 'n02097474', 'n02097658', 'n02098105', 'n02098286', 'n02098413', 'n02099267', 'n02099429', 'n02099601', 'n02099712', 'n02099849', 'n02100236', 'n02100583', 'n02100735', 'n02100877', 'n02101006', 'n02101388', 'n02101556', 'n02102040', 'n02102177', 'n02102318', 'n02102480', 'n02102973', 'n02104029', 'n02104365', 'n02105056', 'n02105162', 'n02105251', 'n02105412', 'n02105505', 'n02105641', 'n02105855', 'n02106030', 'n02106166', 'n02106382', 'n02106550', 'n02106662', 'n02107142', 'n02107312', 'n02107574', 'n02107683', 'n02107908', 'n02108000', 'n02108089', 'n02108422', 'n02108551', 'n02108915', 'n02109047', 'n02109525', 'n02109961', 'n02110063', 'n02110185', 'n02110341', 'n02110627', 'n02110806', 'n02110958', 'n02111129', 'n02111277', 'n02111500', 'n02111889', 'n02112018', 'n02112137', 'n02112350', 'n02112706', 'n02113023', 'n02113186', 'n02113624', 'n02113712', 'n02113799', 'n02113978', 'n02114367', 'n02114548', 'n02114712', 'n02114855', 'n02115641', 'n02115913', 'n02116738', 'n02117135', 'n02119022', 'n02119789', 'n02120079', 'n02120505', 'n02123045', 'n02123159', 'n02123394', 'n02123597', 'n02124075', 'n02125311', 'n02127052', 'n02128385', 'n02128757', 'n02128925', 'n02129165', 'n02129604', 'n02130308', 'n02132136', 'n02133161', 'n02134084', 'n02134418', 'n02137549', 'n02138441', 'n02165105', 'n02165456', 'n02167151', 'n02168699', 'n02169497', 'n02172182', 'n02174001', 'n02177972', 'n02190166', 'n02206856', 'n02219486', 'n02226429', 'n02229544', 'n02231487', 'n02233338', 'n02236044', 'n02256656', 'n02259212', 'n02264363', 'n02268443', 'n02268853', 'n02276258', 'n02277742', 'n02279972', 'n02280649', 'n02281406', 'n02281787', 'n02317335', 'n02319095', 'n02321529', 'n02325366', 'n02326432', 'n02328150', 'n02342885', 'n02346627', 'n02356798', 'n02361337', 'n02363005', 'n02364673', 'n02389026', 'n02391049', 'n02395406', 'n02396427', 'n02397096', 'n02398521', 'n02403003', 'n02408429', 'n02410509', 'n02412080', 'n02415577', 'n02417914', 'n02422106', 'n02422699', 'n02423022', 'n02437312', 'n02437616', 'n02441942', 'n02442845', 'n02443114', 'n02443484', 'n02444819', 'n02445715', 'n02447366', 'n02454379', 'n02457408', 'n02480495', 'n02480855', 'n02481823', 'n02483362', 'n02483708', 'n02484975', 'n02486261', 'n02486410', 'n02487347', 'n02488291', 'n02488702', 'n02489166', 'n02490219', 'n02492035', 'n02492660', 'n02493509', 'n02493793', 'n02494079', 'n02497673', 'n02500267', 'n02504013', 'n02504458', 'n02509815', 'n02510455', 'n02514041', 'n02526121', 'n02536864', 'n02606052', 'n02607072', 'n02640242', 'n02641379', 'n02643566', 'n02655020', 'n02666196', 'n02667093', 'n02669723', 'n02672831', 'n02676566', 'n02687172', 'n02690373', 'n02692877', 'n02699494', 'n02701002', 'n02704792', 'n02708093', 'n02727426', 'n02730930', 'n02747177', 'n02749479', 'n02769748', 'n02776631', 'n02777292', 'n02782093', 'n02783161', 'n02786058', 'n02787622', 'n02788148', 'n02790996', 'n02791124', 'n02791270', 'n02793495', 'n02794156', 'n02795169', 'n02797295', 'n02799071', 'n02802426', 'n02804414', 'n02804610', 'n02807133', 'n02808304', 'n02808440', 'n02814533', 'n02814860', 'n02815834', 'n02817516', 'n02823428', 'n02823750', 'n02825657', 'n02834397', 'n02835271', 'n02837789', 'n02840245', 'n02841315', 'n02843684', 'n02859443', 'n02860847', 'n02865351', 'n02869837', 'n02870880', 'n02871525', 'n02877765', 'n02879718', 'n02883205', 'n02892201', 'n02892767', 'n02894605', 'n02895154', 'n02906734', 'n02909870', 'n02910353', 'n02916936', 'n02917067', 'n02927161', 'n02930766', 'n02939185', 'n02948072', 'n02950826', 'n02951358', 'n02951585', 'n02963159', 'n02965783', 'n02966193', 'n02966687', 'n02971356', 'n02974003', 'n02977058', 'n02978881', 'n02979186', 'n02980441', 'n02981792', 'n02988304', 'n02992211', 'n02992529', 'n02999410', 'n03000134', 'n03000247', 'n03000684', 'n03014705', 'n03016953', 'n03017168', 'n03018349', 'n03026506', 'n03028079', 'n03032252', 'n03041632', 'n03042490', 'n03045698', 'n03047690', 'n03062245', 'n03063599', 'n03063689', 'n03065424', 'n03075370', 'n03085013', 'n03089624', 'n03095699', 'n03100240', 'n03109150', 'n03110669', 'n03124043', 'n03124170', 'n03125729', 'n03126707', 'n03127747', 'n03127925', 'n03131574', 'n03133878', 'n03134739', 'n03141823', 'n03146219', 'n03160309', 'n03179701', 'n03180011', 'n03187595', 'n03188531', 'n03196217', 'n03197337', 'n03201208', 'n03207743', 'n03207941', 'n03208938', 'n03216828', 'n03218198', 'n03220513', 'n03223299', 'n03240683', 'n03249569', 'n03250847', 'n03255030', 'n03259280', 'n03271574', 'n03272010', 'n03272562', 'n03290653', 'n03291819', 'n03297495', 'n03314780', 'n03325584', 'n03337140', 'n03344393', 'n03345487', 'n03347037', 'n03355925', 'n03372029', 'n03376595', 'n03379051', 'n03384352', 'n03388043', 'n03388183', 'n03388549', 'n03393912', 'n03394916', 'n03400231', 'n03404251', 'n03417042', 'n03424325', 'n03425413', 'n03443371', 'n03444034', 'n03445777', 'n03445924', 'n03447447', 'n03447721', 'n03450230', 'n03452741', 'n03457902', 'n03459775', 'n03461385', 'n03467068', 'n03476684', 'n03476991', 'n03478589', 'n03481172', 'n03482405', 'n03483316', 'n03485407', 'n03485794', 'n03492542', 'n03494278', 'n03495258', 'n03496892', 'n03498962', 'n03527444', 'n03529860', 'n03530642', 'n03532672', 'n03534580', 'n03535780', 'n03538406', 'n03544143', 'n03584254', 'n03584829', 'n03590841', 'n03594734', 'n03594945', 'n03595614', 'n03598930', 'n03599486', 'n03602883', 'n03617480', 'n03623198', 'n03627232', 'n03630383', 'n03633091', 'n03637318', 'n03642806', 'n03649909', 'n03657121', 'n03658185', 'n03661043', 'n03662601', 'n03666591', 'n03670208', 'n03673027', 'n03676483', 'n03680355', 'n03690938', 'n03691459', 'n03692522', 'n03697007', 'n03706229', 'n03709823', 'n03710193', 'n03710637', 'n03710721', 'n03717622', 'n03720891', 'n03721384', 'n03724870', 'n03729826', 'n03733131', 'n03733281', 'n03733805', 'n03742115', 'n03743016', 'n03759954', 'n03761084', 'n03763968', 'n03764736', 'n03769881', 'n03770439', 'n03770679', 'n03773504', 'n03775071', 'n03775546', 'n03776460', 'n03777568', 'n03777754', 'n03781244', 'n03782006', 'n03785016', 'n03786901', 'n03787032', 'n03788195', 'n03788365', 'n03791053', 'n03792782', 'n03792972', 'n03793489', 'n03794056', 'n03796401', 'n03803284', 'n03804744', 'n03814639', 'n03814906', 'n03825788', 'n03832673', 'n03837869', 'n03838899', 'n03840681', 'n03841143', 'n03843555', 'n03854065', 'n03857828', 'n03866082', 'n03868242', 'n03868863', 'n03871628', 'n03873416', 'n03874293', 'n03874599', 'n03876231', 'n03877472', 'n03877845', 'n03884397', 'n03887697', 'n03888257', 'n03888605', 'n03891251', 'n03891332', 'n03895866', 'n03899768', 'n03902125', 'n03903868', 'n03908618', 'n03908714', 'n03916031', 'n03920288', 'n03924679', 'n03929660', 'n03929855', 'n03930313', 'n03930630', 'n03933933', 'n03935335', 'n03937543', 'n03938244', 'n03942813', 'n03944341', 'n03947888', 'n03950228', 'n03954731', 'n03956157', 'n03958227', 'n03961711', 'n03967562', 'n03970156', 'n03976467', 'n03976657', 'n03977966', 'n03980874', 'n03982430', 'n03983396', 'n03991062', 'n03992509', 'n03995372', 'n03998194', 'n04004767', 'n04005630', 'n04008634', 'n04009552', 'n04019541', 'n04023962', 'n04026417', 'n04033901', 'n04033995', 'n04037443', 'n04039381', 'n04040759', 'n04041544', 'n04044716', 'n04049303', 'n04065272', 'n04067472', 'n04069434', 'n04070727', 'n04074963', 'n04081281', 'n04086273', 'n04090263', 'n04099969', 'n04111531', 'n04116512', 'n04118538', 'n04118776', 'n04120489', 'n04125021', 'n04127249', 'n04131690', 'n04133789', 'n04136333', 'n04141076', 'n04141327', 'n04141975', 'n04146614', 'n04147183', 'n04149813', 'n04152593', 'n04153751', 'n04154565', 'n04162706', 'n04179913', 'n04192698', 'n04200800', 'n04201297', 'n04204238', 'n04204347', 'n04208210', 'n04209133', 'n04209239', 'n04228054', 'n04229816', 'n04235860', 'n04238763', 'n04239074', 'n04243546', 'n04251144', 'n04252077', 'n04252225', 'n04254120', 'n04254680', 'n04254777', 'n04258138', 'n04259630', 'n04263257', 'n04264628', 'n04265275', 'n04266014', 'n04270147', 'n04273569', 'n04275548', 'n04277352', 'n04285008', 'n04286575', 'n04296562', 'n04310018', 'n04311004', 'n04311174', 'n04317175', 'n04325704', 'n04326547', 'n04328186', 'n04330267', 'n04332243', 'n04335435', 'n04336792', 'n04344873', 'n04346328', 'n04347754', 'n04350905', 'n04355338', 'n04355933', 'n04356056', 'n04357314', 'n04366367', 'n04367480', 'n04370456', 'n04371430', 'n04371774', 'n04372370', 'n04376876', 'n04380533', 'n04389033', 'n04392985', 'n04398044', 'n04399382', 'n04404412', 'n04409515', 'n04417672', 'n04418357', 'n04423845', 'n04428191', 'n04429376', 'n04435653', 'n04442312', 'n04443257', 'n04447861', 'n04456115', 'n04458633', 'n04461696', 'n04462240', 'n04465501', 'n04467665', 'n04476259', 'n04479046', 'n04482393', 'n04483307', 'n04485082', 'n04486054', 'n04487081', 'n04487394', 'n04493381', 'n04501370', 'n04505470', 'n04507155', 'n04509417', 'n04515003', 'n04517823', 'n04522168', 'n04523525', 'n04525038', 'n04525305', 'n04532106', 'n04532670', 'n04536866', 'n04540053', 'n04542943', 'n04548280', 'n04548362', 'n04550184', 'n04552348', 'n04553703', 'n04554684', 'n04557648', 'n04560804', 'n04562935', 'n04579145', 'n04579432', 'n04584207', 'n04589890', 'n04590129', 'n04591157', 'n04591713', 'n04592741', 'n04596742', 'n04597913', 'n04599235', 'n04604644', 'n04606251', 'n04612504', 'n04613696', 'n06359193', 'n06596364', 'n06785654', 'n06794110', 'n06874185', 'n07248320', 'n07565083', 'n07579787', 'n07583066', 'n07584110', 'n07590611', 'n07613480', 'n07614500', 'n07615774', 'n07684084', 'n07693725', 'n07695742', 'n07697313', 'n07697537', 'n07711569', 'n07714571', 'n07714990', 'n07715103', 'n07716358', 'n07716906', 'n07717410', 'n07717556', 'n07718472', 'n07718747', 'n07720875', 'n07730033', 'n07734744', 'n07742313', 'n07745940', 'n07747607', 'n07749582', 'n07753113', 'n07753275', 'n07753592', 'n07754684', 'n07760859', 'n07768694', 'n07802026', 'n07831146', 'n07836838', 'n07860988', 'n07871810', 'n07873807', 'n07875152', 'n07880968', 'n07892512', 'n07920052', 'n07930864', 'n07932039', 'n09193705', 'n09229709', 'n09246464', 'n09256479', 'n09288635', 'n09332890', 'n09399592', 'n09421951', 'n09428293', 'n09468604', 'n09472597', 'n09835506', 'n10148035', 'n10565667', 'n11879895', 'n11939491', 'n12057211', 'n12144580', 'n12267677', 'n12620546', 'n12768682', 'n12985857', 'n12998815', 'n13037406', 'n13040303', 'n13044778', 'n13052670', 'n13054560', 'n13133613', 'n15075141']


# Imagenet - A: Dataloader

In [ ]:
imagenet_a_wnids = ['n01498041', 'n01531178', 'n01534433', 'n01558993', 'n01580077', 'n01614925', 'n01616318', 'n01631663', 'n01641577', 'n01669191', 'n01677366', 
                    'n01687978', 'n01694178', 'n01698640', 'n01735189', 'n01770081', 'n01770393', 'n01774750', 'n01784675', 'n01819313', 'n01820546', 'n01833805', 
                    'n01843383', 'n01847000', 'n01855672', 'n01882714', 'n01910747', 'n01914609', 'n01924916', 'n01944390', 'n01985128', 'n01986214', 'n02007558', 
                    'n02009912', 'n02037110', 'n02051845', 'n02077923', 'n02085620', 'n02099601', 'n02106550', 'n02106662', 'n02110958', 'n02119022', 'n02123394', 
                    'n02127052', 'n02129165', 'n02133161', 'n02137549', 'n02165456', 'n02174001', 'n02177972', 'n02190166', 'n02206856', 'n02219486', 'n02226429', 
                    'n02231487', 'n02233338', 'n02236044', 'n02259212', 'n02268443', 'n02279972', 'n02280649', 'n02281787', 'n02317335', 'n02325366', 'n02346627', 
                    'n02356798', 'n02361337', 'n02410509', 'n02445715', 'n02454379', 'n02486410', 'n02492035', 'n02504458', 'n02655020', 'n02669723', 'n02672831', 
                    'n02676566', 'n02690373', 'n02701002', 'n02730930', 'n02777292', 'n02782093', 'n02787622', 'n02793495', 'n02797295', 'n02802426', 'n02814860', 
                    'n02815834', 'n02837789', 'n02879718', 'n02883205', 'n02895154', 'n02906734', 'n02948072', 'n02951358', 'n02980441', 'n02992211', 'n02999410', 
                    'n03014705', 'n03026506', 'n03124043', 'n03125729', 'n03187595', 'n03196217', 'n03223299', 'n03250847', 'n03255030', 'n03291819', 'n03325584', 
                    'n03355925', 'n03384352', 'n03388043', 'n03417042', 'n03443371', 'n03444034', 'n03445924', 'n03452741', 'n03483316', 'n03584829', 'n03590841', 
                    'n03594945', 'n03617480', 'n03666591', 'n03670208', 'n03717622', 'n03720891', 'n03721384', 'n03724870', 'n03775071', 'n03788195', 'n03804744', 
                    'n03837869', 'n03840681', 'n03854065', 'n03888257', 'n03891332', 'n03935335', 'n03982430', 'n04019541', 'n04033901', 'n04039381', 'n04067472', 
                    'n04086273', 'n04099969', 'n04118538', 'n04131690', 'n04133789', 'n04141076', 'n04146614', 'n04147183', 'n04179913', 'n04208210', 'n04235860', 
                    'n04252077', 'n04252225', 'n04254120', 'n04270147', 'n04275548', 'n04310018', 'n04317175', 'n04344873', 'n04347754', 'n04355338', 'n04366367', 
                    'n04376876', 'n04389033', 'n04399382', 'n04442312', 'n04456115', 'n04482393', 'n04507155', 'n04509417', 'n04532670', 'n04540053', 'n04554684', 
                    'n04562935', 'n04591713', 'n04606251', 'n07583066', 'n07695742', 'n07697313', 'n07697537', 'n07714990', 'n07718472', 'n07720875', 'n07734744', 
                    'n07749582', 'n07753592', 'n07760859', 'n07768694', 'n07831146', 'n09229709', 'n09246464', 'n09472597', 'n09835506', 'n11879895', 'n12057211', 
                    'n12144580', 'n12267677']

imagenet_a_mask = [wnid in set(imagenet_a_wnids) for wnid in all_wnids]

imagenet_a = dset.ImageFolder(root=PATH_TO_IMAGENET_A, transform=test_transform)
imagenet_a_loader = torch.utils.data.DataLoader(imagenet_a, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Imagenet - O: Dataloader

In [ ]:
# Shell script to convert ImageNet 1k validation set to ImageFolder
!wget https://raw.githubusercontent.com/soumith/imagenetloader.torch/master/valprep.sh
!mv ./valprep.sh imagenet_val
%cd imagenet_val
!bash valprep.sh
%cd ..
!pwd

In [ ]:
imagenet_o_wnids = ['n01443537', 'n01704323', 'n01770081', 'n01784675', 'n01819313', 'n01820546', 'n01910747', 'n01917289', 'n01968897', 'n02074367', 'n02317335', 
                    'n02319095', 'n02395406', 'n02454379', 'n02606052', 'n02655020', 'n02666196', 'n02672831', 'n02730930', 'n02777292', 'n02783161', 'n02786058', 
                    'n02787622', 'n02791270', 'n02808304', 'n02817516', 'n02841315', 'n02865351', 'n02877765', 'n02892767', 'n02906734', 'n02910353', 'n02916936', 
                    'n02948072', 'n02965783', 'n03000134', 'n03000684', 'n03017168', 'n03026506', 'n03032252', 'n03075370', 'n03109150', 'n03126707', 'n03134739', 
                    'n03160309', 'n03196217', 'n03207743', 'n03218198', 'n03223299', 'n03240683', 'n03271574', 'n03291819', 'n03297495', 'n03314780', 'n03325584', 
                    'n03344393', 'n03347037', 'n03372029', 'n03376595', 'n03388043', 'n03388183', 'n03400231', 'n03445777', 'n03457902', 'n03467068', 'n03482405', 
                    'n03483316', 'n03494278', 'n03530642', 'n03544143', 'n03584829', 'n03590841', 'n03598930', 'n03602883', 'n03649909', 'n03661043', 'n03666591', 
                    'n03676483', 'n03692522', 'n03706229', 'n03717622', 'n03720891', 'n03721384', 'n03724870', 'n03729826', 'n03733131', 'n03733281', 'n03742115', 
                    'n03786901', 'n03788365', 'n03794056', 'n03804744', 'n03814639', 'n03814906', 'n03825788', 'n03840681', 'n03843555', 'n03854065', 'n03857828', 
                    'n03868863', 'n03874293', 'n03884397', 'n03891251', 'n03908714', 'n03920288', 'n03929660', 'n03930313', 'n03937543', 'n03942813', 'n03944341', 
                    'n03961711', 'n03970156', 'n03982430', 'n03991062', 'n03995372', 'n03998194', 'n04005630', 'n04023962', 'n04033901', 'n04040759', 'n04067472', 
                    'n04074963', 'n04116512', 'n04118776', 'n04125021', 'n04127249', 'n04131690', 'n04141975', 'n04153751', 'n04154565', 'n04201297', 'n04204347', 
                    'n04209133', 'n04209239', 'n04228054', 'n04235860', 'n04243546', 'n04252077', 'n04254120', 'n04258138', 'n04265275', 'n04270147', 'n04275548', 
                    'n04330267', 'n04332243', 'n04336792', 'n04347754', 'n04371430', 'n04371774', 'n04372370', 'n04376876', 'n04409515', 'n04417672', 'n04418357', 
                    'n04423845', 'n04429376', 'n04435653', 'n04442312', 'n04482393', 'n04501370', 'n04507155', 'n04525305', 'n04542943', 'n04554684', 'n04557648', 
                    'n04562935', 'n04579432', 'n04591157', 'n04597913', 'n04599235', 'n06785654', 'n06874185', 'n07615774', 'n07693725', 'n07695742', 'n07697537', 
                    'n07711569', 'n07714990', 'n07715103', 'n07716358', 'n07717410', 'n07718472', 'n07720875', 'n07742313', 'n07745940', 'n07747607', 'n07749582', 
                    'n07753275', 'n07753592', 'n07754684', 'n07768694', 'n07836838', 'n07871810', 'n07873807', 'n07880968', 'n09229709', 'n09472597', 'n12144580', 
                    'n12267677', 'n13052670']

imagenet_o_mask = [wnid in set(imagenet_o_wnids) for wnid in all_wnids]

imagenet_o = dset.ImageFolder(root=PATH_TO_IMAGENET_O, transform=test_transform)
imagenet_o_loader = torch.utils.data.DataLoader(imagenet_o, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
val_imagenet_o = dset.ImageFolder(root=PATH_TO_IMAGENET_VAL, transform=test_transform)
val_imagenet_o_loader = torch.utils.data.DataLoader(val_imagenet_o, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)


# Imagenet - R: Dataloder


In [ ]:
imagenet_r_wnids = {'n01443537', 'n01484850', 'n01494475', 'n01498041', 'n01514859', 'n01518878', 'n01531178', 'n01534433', 'n01614925', 'n01616318', 'n01630670', 
                    'n01632777', 'n01644373', 'n01677366', 'n01694178', 'n01748264', 'n01770393', 'n01774750', 'n01784675', 'n01806143', 'n01820546', 'n01833805', 
                    'n01843383', 'n01847000', 'n01855672', 'n01860187', 'n01882714', 'n01910747', 'n01944390', 'n01983481', 'n01986214', 'n02007558', 'n02009912', 
                    'n02051845', 'n02056570', 'n02066245', 'n02071294', 'n02077923', 'n02085620', 'n02086240', 'n02088094', 'n02088238', 'n02088364', 'n02088466', 
                    'n02091032', 'n02091134', 'n02092339', 'n02094433', 'n02096585', 'n02097298', 'n02098286', 'n02099601', 'n02099712', 'n02102318', 'n02106030', 
                    'n02106166', 'n02106550', 'n02106662', 'n02108089', 'n02108915', 'n02109525', 'n02110185', 'n02110341', 'n02110958', 'n02112018', 'n02112137', 
                    'n02113023', 'n02113624', 'n02113799', 'n02114367', 'n02117135', 'n02119022', 'n02123045', 'n02128385', 'n02128757', 'n02129165', 'n02129604', 
                    'n02130308', 'n02134084', 'n02138441', 'n02165456', 'n02190166', 'n02206856', 'n02219486', 'n02226429', 'n02233338', 'n02236044', 'n02268443', 
                    'n02279972', 'n02317335', 'n02325366', 'n02346627', 'n02356798', 'n02363005', 'n02364673', 'n02391049', 'n02395406', 'n02398521', 'n02410509', 
                    'n02423022', 'n02437616', 'n02445715', 'n02447366', 'n02480495', 'n02480855', 'n02481823', 'n02483362', 'n02486410', 'n02510455', 'n02526121', 
                    'n02607072', 'n02655020', 'n02672831', 'n02701002', 'n02749479', 'n02769748', 'n02793495', 'n02797295', 'n02802426', 'n02808440', 'n02814860', 
                    'n02823750', 'n02841315', 'n02843684', 'n02883205', 'n02906734', 'n02909870', 'n02939185', 'n02948072', 'n02950826', 'n02951358', 'n02966193', 
                    'n02980441', 'n02992529', 'n03124170', 'n03272010', 'n03345487', 'n03372029', 'n03424325', 'n03452741', 'n03467068', 'n03481172', 'n03494278', 
                    'n03495258', 'n03498962', 'n03594945', 'n03602883', 'n03630383', 'n03649909', 'n03676483', 'n03710193', 'n03773504', 'n03775071', 'n03888257', 
                    'n03930630', 'n03947888', 'n04086273', 'n04118538', 'n04133789', 'n04141076', 'n04146614', 'n04147183', 'n04192698', 'n04254680', 'n04266014', 
                    'n04275548', 'n04310018', 'n04325704', 'n04347754', 'n04389033', 'n04409515', 'n04465501', 'n04487394', 'n04522168', 'n04536866', 'n04552348', 
                    'n04591713', 'n07614500', 'n07693725', 'n07695742', 'n07697313', 'n07697537', 'n07714571', 'n07714990', 'n07718472', 'n07720875', 'n07734744', 
                    'n07742313', 'n07745940', 'n07749582', 'n07753275', 'n07753592', 'n07768694', 'n07873807', 'n07880968', 'n07920052', 'n09472597', 'n09835506', 
                    'n10565667', 'n12267677'}

imagenet_r_mask = [wnid in imagenet_r_wnids for wnid in all_wnids]

imagenet_r = dset.ImageFolder(root=PATH_TO_IMAGENET_R, transform=test_transform)
imagenet_r_loader = torch.utils.data.DataLoader(imagenet_r, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Imagenet Sketch, Style: Dataloader

In [ ]:
# !gdown https://drive.google.com/open?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA
# !tar -xvf  './imagenet-sketch.zip

# PATH_TO_IMAGENET_SKETCH = "./imagenet-sketch"

imagenet_sketch = dset.ImageFolder(root=PATH_TO_IMAGENET_SKETCH, transform=test_transform)
sketch_loader = torch.utils.data.DataLoader(imagenet_sketch, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

imagenet_style = dset.ImageFolder(root=PATH_TO_IMAGENET_STYLE, transform=test_transform)
style_loader = torch.utils.data.DataLoader(imagenet_style, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Example

In [ ]:
# Evaluatio functions

concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to('cpu').numpy()

def get_predictions(loader, net=None, mask=None, num_cls=1000):
    net = timm.create_model(net, pretrained=True, num_classes=num_cls)
    net.cuda()
    net.eval()
    confidence = []
    correct = []
    num_correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.cuda(), target.cuda()
            output = net(data)[:,mask]

            # accuracy
            pred = output.data.max(1)[1]
            num_correct += pred.eq(target.data).sum().item()

            confidence.extend(to_np(F.softmax(output, dim=1).max(1)[0]).squeeze().tolist())
            pred = output.data.max(1)[1]
            correct.extend(pred.eq(target).to('cpu').numpy().squeeze().tolist())

    return np.array(confidence), np.array(correct), num_correct


def get_imagenet_results(loader, net=None, mask=None, num_cls=1000):
    confidence, correct, num_correct = get_predictions(loader, net, mask, num_cls)
    acc = num_correct / len(loader.dataset)
    print('Accuracy (%):', round(100*acc, 2))
    show_calibration_results(confidence, correct)
    
    return acc, confidence.copy(), correct.copy()

# ImageNet -O uses AUPR as the evaluation metric, calculating the area under the precision-recall curve.
def get_imagenet_o_results(in_loader, out_loader, net, mask, num_cls=1000):
    confidence_in, correct, num_correct = get_predictions(in_loader, net=net, mask=mask, num_cls=num_cls)
    in_score = -confidence_in
    confidence_out, correct_out, num_correct_out = get_predictions(out_loader, net=net, mask=mask, num_cls=num_cls)
    out_score = -confidence_out

    aurocs, auprs, fprs = [], [], []
    # Calculate the AUPR by out score and in score
    measures = get_measures(out_score, in_score)
    aurocs = measures[0]; auprs = measures[1]; fprs = measures[2]

    print_measures_old(aurocs, auprs, fprs, method_name='MSP')

    # acc = num_correct_out / len(nae_loader.dataset)
    # print('Out Dist Accuracy (%):', round(100*acc, 4))
    print('AUPR (%):', round(100*auprs, 2))
    return auprs.copy(), aurocs.copy(), fprs.copy()


In [ ]:
# Evaluation functions for ImageNet-Sketch and Stylized ImageNet

def get_predictions_style(loader, net=None, num_cls=1000):
    net = timm.create_model(net, pretrained=True, num_classes=num_cls)
    net.cuda()
    net.eval()
    confidence = []
    correct = []
    num_correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.cuda(), target.cuda()
            output = net(data)

            # accuracy
            pred = output.data.max(1)[1]
            num_correct += pred.eq(target.data).sum().item()

            confidence.extend(to_np(F.softmax(output, dim=1).max(1)[0]).squeeze().tolist())
            pred = output.data.max(1)[1]
            correct.extend(pred.eq(target).to('cpu').numpy().squeeze().tolist())

    return np.array(confidence), np.array(correct), num_correct

def get_imagenet_style_results(loader, net=None, num_cls=1000):
    confidence, correct, num_correct = get_predictions_style(loader, net, num_cls)
    acc = num_correct / len(loader.dataset)
    print('Accuracy (%):', round(100*acc, 2))
    show_calibration_results(confidence, correct)
    
    return acc, confidence.copy(), correct.copy()

# Recipe - 1 Running - CNN Model

In [ ]:
cnn_model_lst_all = ['cspresnet50', 'densenet121', 'dla34', 'dpn68', 'efficientnetv2_rw_m',
                     'efficientnet_b0', 'mixnet_l', 'mnasnet_100', 'fbnetc_100', 'spnasnet_100',
                     'tinynet_a', 'ghostnet_100', 'hrnet_w18', 'inception_resnet_v2', 'inception_v3', 'nasnetalarge',
                     'nf_resnet50', 'pnasnet5large', 'res2net50_14w_8s', 'resnest14d', 'resnet10t', 'rexnet_100',
                     'selecsls42b', 'skresnet18', 'tnt_s_patch16_224', 'vgg19', 'ese_vovnet19b_dw', 'xception']

In [ ]:
cnn_accuracy_r = []

for name in cnn_model_lst_all:
  acc_r, test_confidence_r, test_correct_r = get_imagenet_results(imagenet_r_loader, net=name, mask=imagenet_r_mask, num_cls=1000)
  value_r = f'ImageNet-R Accuracy (%) for {name} model: {round(100*acc_r, 2)}'
  cnn_accuracy_r.append(value_r)

print(cnn_accuracy_r)

In [ ]:
cnn_accuracy_o = []

for name in cnn_model_lst_all:
  auprs, aurocs, fpr_list = get_imagenet_o_results(val_imagenet_o_loader, imagenet_o_loader, net=name, mask=imagenet_o_mask, num_cls=1000)
  value = f'ImageNet-O AUPR (%) for {name} model: {round(100*auprs, 2)}'
  cnn_accuracy_o.append(value)
  
print(cnn_accuracy_o)

In [ ]:
cnn_accuracy_o

['ImageNet-O AUPR (%) for cspresnet50 model: 2.19',
 'ImageNet-O AUPR (%) for densenet121 model: 2.17',
 'ImageNet-O AUPR (%) for dla34 model: 2.22',
 'ImageNet-O AUPR (%) for dpn68 model: 2.23',
 'ImageNet-O AUPR (%) for efficientnetv2_rw_m model: 2.21',
 'ImageNet-O AUPR (%) for efficientnet_b0 model: 2.18',
 'ImageNet-O AUPR (%) for mixnet_l model: 2.17',
 'ImageNet-O AUPR (%) for mnasnet_100 model: 2.16',
 'ImageNet-O AUPR (%) for fbnetc_100 model: 2.16',
 'ImageNet-O AUPR (%) for spnasnet_100 model: 2.18',
 'ImageNet-O AUPR (%) for tinynet_a model: 2.18',
 'ImageNet-O AUPR (%) for ghostnet_100 model: 2.18',
 'ImageNet-O AUPR (%) for hrnet_w18 model: 2.23',
 'ImageNet-O AUPR (%) for inception_resnet_v2 model: 2.25',
 'ImageNet-O AUPR (%) for inception_v3 model: 2.29',
 'ImageNet-O AUPR (%) for nasnetalarge model: 2.97',
 'ImageNet-O AUPR (%) for nf_resnet50 model: 2.17',
 'ImageNet-O AUPR (%) for pnasnet5large model: 2.34',
 'ImageNet-O AUPR (%) for res2net50_14w_8s model: 2.11',
 

In [ ]:
cnn_accuracy_style = []

for name in cnn_model_lst_all:
  acc, confidence, correct = get_imagenet_style_results(style_loader, net=name, num_cls=1000)
  value = f'ImageNet-Style Accuracy (%) for {name} model: {round(100*acc, 2)}'
  cnn_accuracy_style.append(value)
  
print(cnn_accuracy_style)

Accuracy (%): 10.58
RMS Calib Error (%): 	26.12
AURRA (%): 		20.75


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/densenet121_ra-50efcf5c.pth" to /root/.cache/torch/hub/checkpoints/densenet121_ra-50efcf5c.pth


Accuracy (%): 8.13
RMS Calib Error (%): 	19.66
AURRA (%): 		14.80


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/dla34-2b83ff04.pth" to /root/.cache/torch/hub/checkpoints/dla34-2b83ff04.pth


Accuracy (%): 7.61
RMS Calib Error (%): 	28.50
AURRA (%): 		14.49


...

Accuracy (%): 7.63
RMS Calib Error (%): 	26.05
AURRA (%): 		13.13


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/fbnetc_100-c345b898.pth" to /root/.cache/torch/hub/checkpoints/fbnetc_100-c345b898.pth


Accuracy (%): 7.66
RMS Calib Error (%): 	29.28
AURRA (%): 		13.24


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/spnasnet_100-048bc3f4.pth" to /root/.cache/torch/hub/checkpoints/spnasnet_100-048bc3f4.pth


Accuracy (%): 7.39
RMS Calib Error (%): 	22.66
AURRA (%): 		13.69


# Recipe - 2 - Running - Vision Transformer Model

In [ ]:
vit_model_lst_all = ['beit_base_patch16_224', 'cait_s24_224', 'coatnet_0_rw_224', 'crossvit_base_240', 
                 'deit_base_patch16_224', 'deit3_base_patch16_224', 'efficientformer_l1', 'gcvit_base',
                 'maxvit_tiny_rw_224', 'mobilevit_s', 'mvitv2_base', 'jx_nest_small', 'pit_b_224'
                 'pit_s_224', 'swin_s3_base_224', 'swinv2_cr_small_224', 'tnt_s_patch16_224', 'twins_pcpvt_base',
                 'visformer_small', 'vit_base_patch32_224', 'volo_d1_224', 'xcit_small_12_p8_224']

In [ ]:
vit_accuracy_r = []

for name in vit_model_lst_all:
  acc_r, test_confidence_r, test_correct_r = get_imagenet_results(imagenet_r_loader, net=name, mask=imagenet_r_mask, num_cls=1000)
  value_r = f'ImageNet-R Accuracy (%) for {name} model: {round(100*acc_r, 2)}'
  vit_accuracy_r.append(value_r)

print(vit_accuracy_r)

In [ ]:
vit_accuracy_o = []

for name in vit_model_lst_all:
  auprs, aurocs, fpr_list = get_imagenet_o_results(val_imagenet_o_loader, imagenet_o_loader, net=name, mask=imagenet_o_mask, num_cls=1000)
  value = f'ImageNet-O AUPR (%) for {name} model: {round(100*auprs, 2)}'
  vit_accuracy_o.append(value)

print(vit_accuracy_o)

In [ ]:
vit_accuracy_style = []

for name in vit_model_lst_all:
  acc, confidence, correct = get_imagenet_style_results(style_loader, net=name, num_cls=1000)
  value = f'ImageNet-Style Accuracy (%) for {name} model: {round(100*acc, 2)}'
  vit_accuracy_style.append(value)
  
print(vit_accuracy_style)

# Recipe - 3 - Combination of CNN and Vision Transformer

In [ ]:
combine_model_lst = ['convnext_base', 'convit_base', 'edgenext_base', 'levit_128']

In [ ]:
combine_accuracy = []

for name in combine_model_lst:
  acc, test_confidence, test_correct = get_imagenet_results(imagenet_r_loader, net=name, mask=imagenet_r_mask, num_cls=1000)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 2)}'
  combine_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth


Accuracy (%): 51.7067
RMS Calib Error (%): 	11.06
AURRA (%): 		79.11


Downloading: "https://dl.fbaipublicfiles.com/convit/convit_base.pth" to /root/.cache/torch/hub/checkpoints/convit_base.pth


Accuracy (%): 48.7867
RMS Calib Error (%): 	4.41
AURRA (%): 		78.03


Downloading: "https://github.com/mmaaz60/EdgeNeXt/releases/download/v1.2/edgenext_base_usi.pth" to /root/.cache/torch/hub/checkpoints/edgenext_base_usi.pth


Accuracy (%): 51.0433
RMS Calib Error (%): 	7.34
AURRA (%): 		79.37


Downloading: "https://dl.fbaipublicfiles.com/LeViT/LeViT-128-b88c2750.pth" to /root/.cache/torch/hub/checkpoints/LeViT-128-b88c2750.pth


Accuracy (%): 40.2833
RMS Calib Error (%): 	14.30
AURRA (%): 		69.47


In [ ]:
combine_accuracy

['ImageNet-A Accuracy (%) for convnext_base model: 51.7067',
 'ImageNet-A Accuracy (%) for convit_base model: 48.7867',
 'ImageNet-A Accuracy (%) for edgenext_base model: 51.0433',
 'ImageNet-A Accuracy (%) for levit_128 model: 40.2833']

# Recipe - 4 - Running MLP Mixer

In [ ]:
mlp_model_lst = ['mixer_b16_224']

In [ ]:
mlp_accuracy = []

for name in mlp_model_lst:
  acc, test_confidence, test_correct = get_imagenet_results(imagenet_r_loader, net=name, mask=imagenet_r_mask, num_cls=1000)
  value = f'ImageNet-R Accuracy (%) for {name} model: {round(100*acc, 2)}'
  mlp_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Accuracy (%): 32.75
RMS Calib Error (%): 	25.04
AURRA (%): 		61.50


In [ ]:
mlp_accuracy

['ImageNet-A Accuracy (%) for mixer_b16_224 model: 32.75']

# 5.    Recipe - 5 - Mobilenet:

In [ ]:
mobilenet_model_lst = ['mobilenetv3_large_100']

In [ ]:
mobilenet_accuracy = []

for name in mobilenet_model_lst:
  acc, test_confidence, test_correct = get_imagenet_results(imagenet_r_loader, net=name, mask=imagenet_r_mask, num_cls=1000)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 2)}'
  mobilenet_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /root/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


Accuracy (%): 35.1067
RMS Calib Error (%): 	8.45
AURRA (%): 		62.61


In [ ]:
mobilenet_accuracy

['ImageNet-A Accuracy (%) for mobilenetv3_large_100 model: 35.1067']

# 6.  Recipe - 6 - Others

In [ ]:
others_model_lst = ['regnetv_040', 'sequencer2d_l']

In [ ]:
others_accuracy_a = []

for name in others_model_lst:
  acc, test_confidence, test_correct = get_imagenet_results(imagenet_r_loader, net=name, mask=imagenet_r_mask, num_cls=1000)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 2)}'
  others_accuracy_a.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

print(others_accuracy_a)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tpu-weights/regnetv_040_ra3-c248f51f.pth" to /root/.cache/torch/hub/checkpoints/regnetv_040_ra3-c248f51f.pth


Accuracy (%): 44.7433
RMS Calib Error (%): 	5.06
AURRA (%): 		73.64


Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth


Accuracy (%): 48.5567
RMS Calib Error (%): 	7.92
AURRA (%): 		76.98


In [ ]:
others_model_lst = ['regnetv_040', 'sequencer2d_l']

others_accuracy_o = []

for name in others_model_lst:
  auprs, aurocs, fpr_list = get_imagenet_o_results(val_imagenet_o_loader, imagenet_o_loader, net=name, mask=imagenet_o_mask, num_cls=1000)
  value = f'ImageNet-O AUPR (%) for {name} model: {round(100*auprs, 2)}'
  others_accuracy_o.append(value)

print(others_accuracy_o)

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 37381
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37386
  warnings.warn(


			MSP
FPR95:	98.67
AUROC: 	14.36
AUPR:  	2.17


Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 37381
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37386
  warnings.warn(


			MSP
FPR95:	98.67
AUROC: 	14.53
AUPR:  	2.18


['ImageNet-O AUPR (%) for regnetv_040 model: 2.17',
 'ImageNet-O AUPR (%) for sequencer2d_l model: 2.18']

In [ ]:
others_model_lst = ['regnetv_040', 'sequencer2d_l']

others_accuracy_style = []

for name in others_model_lst:
  acc, confidence, correct = get_imagenet_style_results(style_loader, net=name, num_cls=1000)
  value = f'ImageNet-Style Accuracy (%) for {name} model: {round(100*acc, 2)}'
  others_accuracy_style.append(value)

print(others_accuracy_style)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tpu-weights/regnetv_040_ra3-c248f51f.pth" to /root/.cache/torch/hub/checkpoints/regnetv_040_ra3-c248f51f.pth


Accuracy (%): 11.88
RMS Calib Error (%): 	21.39
AURRA (%): 		23.21


Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth


Accuracy (%): 16.74
RMS Calib Error (%): 	21.59
AURRA (%): 		30.03


['ImageNet-Style Accuracy (%) for regnetv_040 model: 11.88',
 'ImageNet-Style Accuracy (%) for sequencer2d_l model: 16.74']